In [1]:
import requests

seasons = [str(s) for s in range(2022,2023)]


In [4]:
base_url = "https://www.hockey-reference.com"
schedule_links = [f"{base_url}/leagues/NHL_{season}_games.html" for season in seasons]


429
https://www.hockey-reference.com/

<!DOCTYPE html>
<html data-version="klecko-" data-root="" lang="en" class="no-js" >
<head id="suppress_all_ads">
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />

    <title>Rate Limited Request (429 error) | Sports-Reference.com</title>

    <link rel="canonical" href="https://www.sports-reference.com/429.html" />

<!-- CSS start -->
 <style>body,html{color:#000;f


In [3]:
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
from io import StringIO
from bs4 import BeautifulSoup
import time

all_games = []
all_box_scores_links = []
for season in schedule_links:
    data = requests.get(season)   # gets html page for a season
    soup = BeautifulSoup(data.text)
    data = StringIO(data.text)    # Turn raw HTML into file-like object
    games = pd.read_html(data, match='NHL Regular Season Schedule')
    games = games[0]              # Dataframe
    all_games.append(games)
    box_scores_table = soup.find(id='games')  # Get table for box scores links
    box_scores_links = [l.get("href") for l in box_scores_table.find_all("a")]    # Get all links
    box_scores_links = [f"{base_url}{l}" for l in box_scores_links if l and "boxscores/" in l]   # Get absolute URLs
    all_box_scores_links.extend(box_scores_links)
    time.sleep(3)

games = pd.concat(all_games)

ValueError: No tables found

In [54]:
games = games.drop(['LOG', 'Unnamed: 6', 'Notes'], axis=1)    # dropping non-relevant columns
games['Home Team Win'] = games['Home'] > games['Visitor']     # creating win column
games['Home Team Win'] = games['Home Team Win'].astype(int)   # Converting all True/False to 1/0

In [55]:
games

,Date,Time,Visitor,G,Home,G.1,Att.,Home Team Win
0,2021-10-12,7:30 PM,Pittsburgh Penguins,6,Tampa Bay Lightning,2,19092.0,1
1,2021-10-12,10:00 PM,Seattle Kraken,3,Vegas Golden Knights,4,18431.0,1
2,2021-10-13,10:00 PM,Winnipeg Jets,1,Anaheim Ducks,4,16260.0,0
3,2021-10-13,10:00 PM,Chicago Blackhawks,2,Colorado Avalanche,4,18037.0,1
4,2021-10-13,10:00 PM,Vancouver Canucks,2,Edmonton Oilers,3,16034.0,0
...,...,...,...,...,...,...,...,...
1307,2022-04-29,10:00 PM,San Jose Sharks,0,Seattle Kraken,3,17151.0,1
1308,2022-04-29,8:00 PM,Vegas Golden Knights,7,St. Louis Blues,4,18096.0,0
1309,2022-04-29,7:00 PM,Boston Bruins,2,Toronto Maple Leafs,5,18219.0,1
1310,2022-04-29,8:00 PM,Calgary Flames,1,Winnipeg Jets,3,14202.0,1


In [56]:
#all_box_scores_links
print(f"Length of box_scores_links: {len(all_box_scores_links)}")
print(f"Length of games: {len(games)}")

Length of box_scores_links: 1312
Length of games: 1312


In [57]:
print(all_box_scores_links)

['https://www.hockey-reference.com/boxscores/202110120TBL.html', 'https://www.hockey-reference.com/boxscores/202110120VEG.html', 'https://www.hockey-reference.com/boxscores/202110130ANA.html', 'https://www.hockey-reference.com/boxscores/202110130COL.html', 'https://www.hockey-reference.com/boxscores/202110130EDM.html', 'https://www.hockey-reference.com/boxscores/202110130TOR.html', 'https://www.hockey-reference.com/boxscores/202110130WSH.html', 'https://www.hockey-reference.com/boxscores/202110140BUF.html', 'https://www.hockey-reference.com/boxscores/202110140CAR.html', 'https://www.hockey-reference.com/boxscores/202110140CBJ.html', 'https://www.hockey-reference.com/boxscores/202110140DET.html', 'https://www.hockey-reference.com/boxscores/202110140FLA.html', 'https://www.hockey-reference.com/boxscores/202110140LAK.html', 'https://www.hockey-reference.com/boxscores/202110140NSH.html', 'https://www.hockey-reference.com/boxscores/202110140NYR.html', 'https://www.hockey-reference.com/boxsc

In [61]:
games.head()
print(all_box_scores_links[78])


https://www.hockey-reference.com/boxscores/202110240BOS.html


In [60]:
box_scores_stats = []

links = all_box_scores_links[78]
#for links in all_box_scores_links:

data = requests.get(links)
data = StringIO(data.text)
v_stats_table = pd.read_html(data, header=1)[2].iloc[[-1]]   # Visitor team stats
h_stats_table = pd.read_html(data, header=1)[4].iloc[[-1]]   # Home team stats

final_v_stats_table = v_stats_table[['PIM', 'S', 'S%']]
final_h_stats_table = h_stats_table[['PIM', 'S', 'S%']]

final_v_stats_table.rename(columns={'PIM' : 'Visitors PIM', 'S' : 'Visitors S', 'S%' : 'Visitors S%'}, inplace=True)
final_h_stats_table.rename(columns={'PIM' : 'Home PIM', 'S' : 'Home S', 'S%' : 'Home S%'}, inplace=True)

final_game_stats = pd.concat([final_v_stats_table, final_h_stats_table], axis=1)

#box_scores_stats.append(final_game_stats)


#final_h_stats_table.rename(columns={'Unnamed: 6_level_0': 'Penalty Minutes', 'Unnamed: 14_level_0': 'SG', 'Unnamed: 15_level_0' : 'S%'}, inplace=True)
#final_h_stats_table
#box_scores_stats




ValueError: No tables found

In [ ]:
#for link in all_box_scores_links:
#    data = requests.get(link)
#    goalie_stats = pd.read_html(data.text, match='Goalies Table')


#link = all_box_scores_links[0]
#data = requests.get(link)
#data = StringIO(data.text)
#v_goalie_stats = pd.read_html(data, match='Goalies Table')[0]   # visitor team's goalie stats
#h_goalie_stats = pd.read_html(data, match='Goalies Table')[1]   # home team's goalie stats